In [118]:
from faker import Faker
# Conexion
import psycopg2
import uuid

# Configuración de la conexion
conexion = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="contraseña"
)

# Consulta para obtener las tablas de un esquema
esquema = 'recruitment'
tablas_db = []
consulta = f"""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = '{esquema}' AND table_type = 'BASE TABLE';
"""

# Ejecutar la consulta
with conexion.cursor() as cursor:
    cursor.execute(consulta)
    tablas = cursor.fetchall()
    tablas_db = tablas

In [119]:

# def recorrer_tabla(tabla):
#     # Consulta para obtener las columnas de una tabla
#     consulta = f"""
#     SELECT column_name, data_type
#     FROM information_schema.columns
#     WHERE table_schema = %s AND table_name = %s;
#     """

#     # Ejecutar la consulta
#     with conexion.cursor() as cursor:
#         cursor.execute(consulta, (esquema, tabla))
#         columnas = cursor.fetchall()

#     # Cerrar la conexión

#     print(tabla)
#     # Mostrar las columnas
#     for columna in columnas:
#         print(f"Columna: {columna[0]}, Tipo de dato: {columna[1]}")

In [120]:
# for tabla in tablas_db:
#     recorrer_tabla(tabla)

In [121]:
# conexion.close()

In [122]:
# for tabla in tablas_db:
#     print(tabla[0])

In [123]:
# Instancia Faker
fake = Faker()
import random

In [124]:
def generar_datos_applicant_profile(num_registros):
    datos = []
    for _ in range(num_registros):
        nombre = fake.name()
        email = fake.email()
        id_usuario = str(uuid.uuid4())
        
        # Aquí puedes añadir otras columnas según el diseño de tu tabla
        datos.append((nombre, email, id_usuario))
    return datos

In [153]:
registros = generar_datos_applicant_profile(200)

In [154]:
registros

[('Steve Walker',
  'chaveztracy@example.org',
  '9899e333-6e4b-4b22-be53-ad284ef14ef2'),
 ('Roberta Cooper',
  'alexisho@example.net',
  'e3bb46eb-bdf8-4f94-8fab-cfd297c8b219'),
 ('Deborah Moore',
  'imiller@example.org',
  'e47b6a04-8f7b-439e-b2c5-7090fce34456'),
 ('Jared Coleman',
  'victorwilliams@example.com',
  '35473ddb-63f9-475d-a52c-7078667e6ef3'),
 ('Barbara Miller',
  'corycastaneda@example.com',
  '2639f0c5-2c0c-4028-8d33-4953bf7580e0'),
 ('Amber Romero',
  'martindenise@example.net',
  '724cef10-265b-4b7e-bec4-bc4c9bf0e5be'),
 ('Olivia Monroe',
  'jeremypittman@example.com',
  'ba7fe009-bf91-4d28-aec6-222d69ef0c4a'),
 ('Ruben Barnes',
  'reginald76@example.net',
  'a5864625-cea4-4f56-9506-fcbd92b7f992'),
 ('Debbie Smith',
  'cferguson@example.org',
  '9971ae2b-5142-4d6d-98e8-08226550084d'),
 ('Shannon Garner',
  'xgonzales@example.org',
  'b0836e4f-63a6-4388-b06b-ec38f1ad02fd'),
 ('Joshua Ramos',
  'hooversteven@example.com',
  '2f7b20a9-4d0d-4028-ba8a-4465deea8efe'),
 ('N

In [155]:
# Insertar los datos generados en la base de datos
def insertar_datos_applicant_profile(datos):
    with conexion.cursor() as cursor:
        # Consulta SQL para insertar registros
        
        consulta = """
        INSERT INTO recruitment.user (name, email, user_id)
        VALUES (%s, %s, %s)
        """
        # Insertar múltiples registros
        cursor.executemany(consulta, datos)
        conexion.commit()  # Confirmar la transacción

In [156]:
insertar_datos_applicant_profile(registros)

In [157]:
import psycopg2

def insertar_user_ids_no_existentes():
    try:
        # Conectar a la base de datos
        with psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        ) as conexion:
            with conexion.cursor() as cursor:
                # Leer los 'user_id' existentes en la tabla 'user'
                consulta_user_ids = """
                SELECT user_id FROM recruitment.user
                """
                cursor.execute(consulta_user_ids)
                user_ids_existentes = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en un set

                # Leer los 'user_id' que ya existen en la tabla 'applicant_profile'
                consulta_applicant_profile = """
                SELECT user_id FROM recruitment.applicant_profile
                """
                cursor.execute(consulta_applicant_profile)
                user_ids_en_profile = set(row[0] for row in cursor.fetchall())  # Almacenar los user_id en applicant_profile

                # Identificar los 'user_id' que están en 'user' pero no en 'applicant_profile'
                user_ids_a_insertar = user_ids_existentes - user_ids_en_profile

                if user_ids_a_insertar:
                    # Insertar los 'user_id' que faltan en la tabla 'applicant_profile'
                    consulta_insertar = """
                    INSERT INTO recruitment.applicant_profile (user_id)
                    VALUES (%s)
                    """
                    cursor.executemany(consulta_insertar, [(user_id,) for user_id in user_ids_a_insertar])
                    conexion.commit()  # Confirmar la transacción

                    print(f"{len(user_ids_a_insertar)} user_ids insertados correctamente en 'applicant_profile'.")
                else:
                    print("No hay nuevos user_ids para insertar en 'applicant_profile'.")
                    return True

    except Exception as e:
        # Imprimir el error en caso de que ocurra una excepción
        print(f"Ocurrió un error: {e}")
        # Si ocurre un error, hacer rollback de cualquier cambio no confirmado
        if 'conexion' in locals():
            conexion.rollback()


In [158]:
while (True):
    if(insertar_user_ids_no_existentes()):
        break

200 user_ids insertados correctamente en 'applicant_profile'.
No hay nuevos user_ids para insertar en 'applicant_profile'.


In [166]:
import psycopg2
import random

def asignar_skills_random_a_applicants():
    try:
        # Conectar a la base de datos
        conexion = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="contraseña"
        )

        with conexion.cursor() as cursor:
            # Obtener todos los applicant_profile_id y sus skills asignados
            consulta_applicant_skills = """
            SELECT applicant_profile_id, COUNT(skill_id) 
            FROM recruitment.applicantprofile_skill
            GROUP BY applicant_profile_id
            """
            cursor.execute(consulta_applicant_skills)
            skills_por_applicant = {row[0]: row[1] for row in cursor.fetchall()}

            # Obtener todos los applicant_profile_id de la tabla 'applicant_profile'
            consulta_applicants = """
            SELECT id FROM recruitment.applicant_profile
            """
            cursor.execute(consulta_applicants)
            applicant_ids = [row[0] for row in cursor.fetchall()]

            # Obtener los skill_id válidos desde la tabla 'skills'
            consulta_skills = """
            SELECT id FROM recruitment.skills
            """
            cursor.execute(consulta_skills)
            skill_id_validos = [row[0] for row in cursor.fetchall()]

            if not skill_id_validos:
                print("No hay skills válidos disponibles en la tabla 'skills'.")
                return

            # Crear combinaciones de applicant_profile_id con nuevos skill_id aleatorios
            registros_a_insertar = []
            for applicant_id in applicant_ids:
                # Verificar si ya tiene 7 o más skills asignados
                skills_actuales = skills_por_applicant.get(applicant_id, 0)
                if skills_actuales >= 7:
                    continue

                # Determinar cuántos skills adicionales se necesitan
                skills_faltantes = 7 - skills_actuales
                if len(skill_id_validos) < skills_faltantes:
                    print(f"No hay suficientes skills válidos para el applicant {applicant_id}.")
                    continue

                # Seleccionar skill_id únicos al azar que aún no estén asignados
                skill_ids_random = random.sample(skill_id_validos, skills_faltantes)
                registros_a_insertar.extend([(applicant_id, skill_id) for skill_id in skill_ids_random])

            # Insertar las combinaciones en la tabla 'applicantprofile_skill'
            if registros_a_insertar:
                consulta_insertar = """
                INSERT INTO recruitment.applicantprofile_skill (applicant_profile_id, skill_id)
                VALUES (%s, %s)
                """
                cursor.executemany(consulta_insertar, registros_a_insertar)
                conexion.commit()
                print(f"Se asignaron nuevos skills a los applicants.")
            else:
                print("No se encontraron applicants que requieran más skills.")

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        if conexion:
            conexion.rollback()
    finally:
        if conexion:
            conexion.close()

# Llamar a la función
asignar_skills_random_a_applicants()


No se encontraron applicants que requieran más skills.
